In [1]:
with open('keys.txt', 'r') as file:
    # Read the entire contents of the file into a variable
    file_contents = file.read()
    keys = file_contents.split('\n')

import praw

reddit = praw.Reddit(
    client_id=keys[0],
    client_secret=keys[1],
    user_agent=keys[2],
    username=keys[3],
    password=keys[4]
)

In [2]:
subreddit = reddit.subreddit('wallstreetbets')
for post in subreddit.new(limit=10):
    print(post.title)

Tesla Stock - What are your main questions around the stock that remain unanswered
Do fertilizers play a role in these increasing pollen allergies? Pharmaceutical companies can make exponential seasonal revenue.
Capital Expenditure Turmoil: Trouble plague financial investment as per the fed survey
Thank you Su bae bae
QQQ PUTS 2 weeks ago took some money out. Now I went from 34k to 68k
This is the way
$AMD $94.25 is the new floor
Thunder calls and thunder puts?
Sure is slippery in here
Breaking News: SAFEBANKING Hearing to take place this Thursday May 11th! US MSOS will easily pop 50%-100% GTBIF TCNNF MSOS


In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

def process_text(text):
    doc = nlp(text)
    return doc

In [4]:
for post in subreddit.new(limit=10):
    doc = process_text(post.title)
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)

Tesla Tesla PROPN NNP compound
Stock Stock PROPN NNP npadvmod
- - PUNCT : punct
What what PRON WP attr
are be AUX VBP ROOT
your your PRON PRP$ poss
main main ADJ JJ amod
questions question NOUN NNS nsubj
around around ADP IN prep
the the DET DT det
stock stock NOUN NN pobj
that that PRON WDT nsubj
remain remain VERB VBP relcl
unanswered unanswered ADJ JJ acomp
Do do AUX VB aux
fertilizers fertilizer NOUN NNS nsubj
play play VERB VB ROOT
a a DET DT det
role role NOUN NN dobj
in in ADP IN prep
these these DET DT det
increasing increase VERB VBG amod
pollen pollen NOUN NN compound
allergies allergy NOUN NNS pobj
? ? PUNCT . punct
Pharmaceutical pharmaceutical ADJ JJ amod
companies company NOUN NNS nsubj
can can AUX MD aux
make make VERB VB ROOT
exponential exponential ADJ JJ amod
seasonal seasonal ADJ JJ amod
revenue revenue NOUN NN dobj
. . PUNCT . punct
Capital Capital PROPN NNP compound
Expenditure Expenditure PROPN NNP compound
Turmoil turmoil NOUN NN ROOT
: : PUNCT : punct
Trouble tr